# UMAP for bird-songs  
---

In [1]:
## Installs for Coursera (on terminal only)
# conda install -c conda-forge librosa umap-learn
# conda install -c plotly plotly=4.14.3

In [2]:
## Installs for CoLab
# !pip install umap-learn
# !pip install plotly==4.14.3

In [3]:
# # For Colab only
# # 4/1AY0e-g6CTocvBK45PWlJu2ycBuOMTgu36b-VZgpnCXitW_Vy3ckpGGBzur8
# from google.colab import drive
# drive.mount('content')
# # /content/content/MyDrive/bird-songs/audio

In [409]:
from umap import UMAP
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

import os
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import pandas as pd
import glob
# import ffmpeg
%matplotlib inline

# Set general font size
plt.rcParams['font.size'] = '14'

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

# Check whether google drive mounted
path = '/content/content/MyDrive/bird-songs/'
if os.path.isdir(path) == True:
    print('Google Drive Mounted')
    run_on_colab = True
else:
    print('Using local drive')
    run_on_colab = False

Using local drive


In [13]:
#@title
# def get_audio_filenames(audio_folder):
#     '''Create a list of audio files in the provided folder'''
    
#     audio_files = []
#     for fn_mp3 in glob.glob(f"{audio_folder}*"):
#         audio_files.append(fn_mp3)
#     return audio_files

In [489]:
SAMPLE_RATE = 22050
NUMBER_MFCC = 32
NUMBER_MEL = 32
F_MIN = 4000 # Minimum Hz
N_FFT = 512 # Choose 2^n where n is integer
HOP_LENGTH = 256 # Choose equal, half, or quarter of N_FFT
NUM_DIVISIONS = 2 # 1, 2, 4, 8

In [490]:
def audio_path_filename(id):
    
    if run_on_colab == True:
        path_filename = '/content/content/MyDrive/bird-songs/audio_8sec/' + str(id) + '.wav'
                                
    else:
        path_filename = 'audio_8sec/' + str(id) + '.wav'
    
    return path_filename


def load_xeno_canto_data():
    '''
    '''

    if run_on_colab == True:
        df = pd.read_csv('/content/content/MyDrive/bird-songs/features_filtered.csv', encoding='latin')
    else:
        df = pd.read_csv('features_filtered.csv', encoding='latin')
    
    df.rename(columns={"id": "fileid"}, inplace=True)
    
    # These are the top species we selected for our analysis
    selected_species = ['Red-winged Blackbird', 'Common Yellowthroat', 'Northern Cardinal', 
                        'Carolina Wren', 'Red Crossbill', 'Spotted Towhee']

    # We only need the id and the labels.
    df = df[df.en.isin(selected_species)][['fileid', 'en']]
    
    # Now we need to change our labels into integers as torch does not support string format for labels.
    labels = {"Red-winged Blackbird": 0, 'Common Yellowthroat': 1, 'Northern Cardinal': 2,
              'Carolina Wren': 3,'Red Crossbill': 4, 'Spotted Towhee': 5}
    df['label'] = df.apply(lambda row: labels[row.en], axis=1)
   
    df['filename'] = df['fileid'].apply(lambda x: audio_path_filename(x))
    
    df.set_index('fileid', inplace=True)
    
    return df


df_xeno_canto_import = load_xeno_canto_data()
df_xeno_canto_import.head(5)

,en,label,filename
fileid,,,
564895,Carolina Wren,3,audio_8sec/564895.wav
545775,Carolina Wren,3,audio_8sec/545775.wav
540857,Carolina Wren,3,audio_8sec/540857.wav
540855,Carolina Wren,3,audio_8sec/540855.wav
539275,Carolina Wren,3,audio_8sec/539275.wav


In [491]:
def find_best_subclip(df, number_divisions=1, sample_rate=22050):
    ''' '''
     
    if number_divisions not in [1,2,4,8]:
        print("Error: You must only select 1, 2, 4, or 8 divisions")
        return pd.DataFrame()
    
    for fileid in tqdm(list(df.index)[:]):
        
        y, sr = librosa.load(df.loc[fileid, 'filename'], sr=sample_rate, mono=True)
        
        # linear rms level and scaling factor
        # https://www.youtube.com/watch?v=zyqb06g51jw
        rms_level_db = 0
        sig = y
        r = 10**(rms_level_db / 20.0)
        a = np.sqrt( (len(sig) * r**2) / np.sum(sig**2) )
        # Normalized amplitude signal
        y_norm = y * a
        
        # fig, ax = plt.subplots(figsize=(8, 3.5))
        # librosa.display.waveplot(y_norm, sr, x_axis='s')
        # plt.ylabel('Amplitude')
       
        # Clip negative amplitude values for area calculation
        y_norm_positive = y_norm.clip(min=0)
        
        # Calculate areas for each window
        length_audio_clip = int(len(y_norm_positive)/sample_rate) # seconds
        duration_per_window = int(length_audio_clip/number_divisions) # seconds
        hop_length = duration_per_window / 10 # seconds
        total_hops = int(length_audio_clip / hop_length)
        # print(f"Audio clip is {length_audio_clip} sec, duration per window is {duration_per_window} sec")
        # print(f"Window hop length is {hop_length} sec, total window hops is {int(total_hops)}\n")
        
        area = []
        for hop in range(total_hops):
            start_window = int(hop*hop_length*sample_rate)
            end_window = int((hop+1)*hop_length*sample_rate) + 1
            # print(f"window {hop} goes from {start_window/sample_rate} to {(end_window-1)/sample_rate}")
            y_window = y_norm_positive[start_window:end_window]
            area_window = np.trapz(y_window, dx = 1/sample_rate, axis=0)
            # print(f"window {hop} positive area under curve is {area_window}")
            area.append(area_window)
            
        
        max_window = np.argmax(area)
        # print(f"Hop window with max area is {max_window}")
        max_start_window = int(max_window*hop_length*sample_rate)
        max_start_window_seconds = max_start_window / sample_rate
        max_end_window = int((max_window+1)*hop_length*sample_rate) + 1
        max_end_window_seconds = (max_end_window-1) / sample_rate
        max_y_window = y_norm_positive[max_start_window:max_end_window]
        # print(f"Max hop window starting time is {max_start_window_seconds}")
        # print(f"Max hop window ending time is {max_end_window_seconds}")
        max_area_window = np.trapz(max_y_window, dx = 1/sample_rate, axis=0)
        # print(f"Max area of window is {max_area_window:.4f}")
        
        
        S, phase = librosa.magphase(librosa.stft(max_y_window))
        rms = librosa.feature.rms(S=S)
        rms_sum = np.sum(rms)
        # print(f"RMS area is {rms_sum:.4f}\n")
        
        # fig, ax = plt.subplots(figsize=(8, 3.5))
        # original_x_ticks =  list(np.arange(0, len(max_y_window) / sample_rate,
        #                         (length_audio_clip / number_divisions)))
        # librosa.display.waveplot(max_y_window, sr, offset = max_start_window_seconds, x_axis='s')
        # plt.ylabel('Amplitude')
        # plt.show()

        df.loc[fileid,'start'] = int(max_start_window)
        df.loc[fileid,'end'] = int(max_end_window)
        df.loc[fileid,'rms'] = rms_sum
  
    return df

df_xeno_canto = find_best_subclip(df=df_xeno_canto_import, number_divisions=NUM_DIVISIONS, 
                                  sample_rate=SAMPLE_RATE)
df_xeno_canto.head(5)

  0%|          | 0/2516 [00:00<?, ?it/s]

,en,label,filename,start,end,rms
fileid,,,,,,
564895,Carolina Wren,3,audio_8sec/564895.wav,70560.0,79381.0,15.446119
545775,Carolina Wren,3,audio_8sec/545775.wav,79380.0,88201.0,17.320824
540857,Carolina Wren,3,audio_8sec/540857.wav,8820.0,17641.0,6.876124
540855,Carolina Wren,3,audio_8sec/540855.wav,158760.0,167581.0,14.775598
539275,Carolina Wren,3,audio_8sec/539275.wav,132300.0,141121.0,12.824493


In [492]:
def create_mfcc_spectral_df_xeno_canto(number_mfcc, sample_rate=22050, best_subclip=False):
    ''' '''

    # Create df to hold mfcc spectral df_xeno_canto
    df = pd.DataFrame(columns = ['mfcc_max' + str(item+1) for item in list(range(number_mfcc))] +
                                ['mfcc_std' + str(item+1) for item in list(range(number_mfcc))], 
                                index=df_xeno_canto.index)
      
    for fileid in tqdm(list(df_xeno_canto.index)[:]):

        y, sr = librosa.load(df_xeno_canto.loc[fileid, 'filename'], sr=sample_rate, mono=True)
       
        # linear rms level and scaling factor
        # https://www.youtube.com/watch?v=zyqb06g51jw
        rms_level_db = 0
        sig = y
        r = 10**(rms_level_db / 20.0)
        a = np.sqrt( (len(sig) * r**2) / np.sum(sig**2) )
        # Normalized amplitude signal
        y_norm = y * a
        
        if best_subclip==True:
            start = int(df_xeno_canto.loc[fileid, 'start'])
            end = int(df_xeno_canto.loc[fileid, 'end'])
            y_norm = y_norm[start:end]
        
        mfccs = librosa.feature.mfcc(y=y_norm, sr=sample_rate, n_mfcc=number_mfcc, n_fft=N_FFT,
                                     hop_length=HOP_LENGTH, fmin=F_MIN)

        mfcc_maxs = [np.max(item) for item in mfccs]
        mfcc_stds = [np.std(item) for item in mfccs]
        # print("mfcc means:", mfcc_maxs, "\nmfcc std deviations:", mfcc_stds)
        df.loc[fileid] = mfcc_maxs + mfcc_stds
  
    return df



df_mfcc = create_mfcc_spectral_df_xeno_canto(NUMBER_MFCC, sample_rate=SAMPLE_RATE, best_subclip=True)
df_mfcc.head()

  0%|          | 0/2516 [00:00<?, ?it/s]

,mfcc_max1,mfcc_max2,mfcc_max3,mfcc_max4,mfcc_max5,mfcc_max6,mfcc_max7,mfcc_max8,mfcc_max9,mfcc_max10,...,mfcc_std23,mfcc_std24,mfcc_std25,mfcc_std26,mfcc_std27,mfcc_std28,mfcc_std29,mfcc_std30,mfcc_std31,mfcc_std32
fileid,,,,,,,,,,,,,,,,,,,,,
564895,-149.43631,85.261353,15.307281,69.582359,19.584253,42.59602,42.389046,25.379915,18.829433,23.294621,...,8.852057,6.502211,7.295641,7.788605,7.366406,7.596004,5.759296,6.563374,5.651157,6.884879
545775,-34.174389,287.955383,19.568802,55.773628,47.664665,19.96027,20.161135,11.108946,18.915283,36.561893,...,9.336739,8.605365,6.666317,6.182525,8.172654,6.63536,8.055593,7.836455,8.866871,6.224747
540857,-307.844696,70.258423,39.420944,10.251158,12.130444,88.087898,68.197449,13.902607,13.910336,17.68701,...,8.253517,10.752787,10.754602,9.494974,5.921476,5.531656,7.236214,6.71984,7.269961,5.448444
540855,-559.795593,8.064632,12.457974,19.946301,16.996418,9.162932,17.693989,31.707882,12.251001,13.17939,...,7.186693,5.950534,6.90855,5.270881,6.071951,6.998231,4.851827,5.825338,6.511818,5.856839
539275,-217.228653,150.939987,180.321899,92.488434,43.013515,30.154858,8.979919,8.44879,8.285357,27.00325,...,8.510853,5.983621,7.962064,8.297771,7.102838,9.243234,8.243326,7.191968,7.967171,6.314699


In [493]:
def create_mel_spectral_features(n_mels=128, sample_rate=22050, best_subclip=False):
    ''' '''

    # Create df to hold mfcc spectral features
    df = pd.DataFrame(columns = ['mel_max' + str(item+1) for item in list(range(n_mels))] +
                                ['mel_std' + str(item+1) for item in list(range(n_mels))], 
                                index=df_xeno_canto.index)
        

    for fileid in tqdm(list(df_xeno_canto.index)[:]):
        

        y, sr = librosa.load(df_xeno_canto.loc[fileid, 'filename'], sr=sample_rate, mono=True)
        
        # linear rms level and scaling factor
        # https://www.youtube.com/watch?v=zyqb06g51jw
        rms_level_db = 0
        sig = y
        r = 10**(rms_level_db / 20.0)
        a = np.sqrt( (len(sig) * r**2) / np.sum(sig**2) )
        # Normalized amplitude signal
        y_norm = y * a
        
        
        if best_subclip==True:
            start = int(df_xeno_canto.loc[fileid, 'start'])
            end = int(df_xeno_canto.loc[fileid, 'end'])
            y_norm = y_norm[start:end]
        
        mels = librosa.feature.melspectrogram(y=y_norm, sr=sample_rate, n_mels=n_mels, 
                                              n_fft=N_FFT, hop_length=HOP_LENGTH, fmin=F_MIN)
        # print(mels.shape)
        mels_maxs = [np.max(item) for item in mels]
        mels_stds = [np.std(item) for item in mels]
        # print("mel means:", mels_maxs, "\nmel std deviations:", mels_stds)
        df.loc[fileid] = mels_maxs + mels_stds
  
    return df


df_mel = create_mel_spectral_features(n_mels=NUMBER_MEL, sample_rate=SAMPLE_RATE, best_subclip=True)
df_mel.head()

  0%|          | 0/2516 [00:00<?, ?it/s]

,mel_max1,mel_max2,mel_max3,mel_max4,mel_max5,mel_max6,mel_max7,mel_max8,mel_max9,mel_max10,...,mel_std23,mel_std24,mel_std25,mel_std26,mel_std27,mel_std28,mel_std29,mel_std30,mel_std31,mel_std32
fileid,,,,,,,,,,,,,,,,,,,,,
564895,0.283351,0.230467,0.189655,0.160028,0.133285,0.11171,0.098598,0.079837,0.066724,0.058691,...,0.056403,0.014043,0.017037,0.021666,0.038108,0.002578,0.002215,0.002122,0.002095,0.002019
545775,1923.399658,1497.391113,1867.647217,418.964478,206.609512,80.128273,55.718712,145.71991,179.215363,44.115192,...,0.051102,0.019693,0.004096,0.00468,0.005511,0.003314,0.003172,0.003048,0.002964,0.002909
540857,0.12915,0.121444,0.031221,0.417643,1.124814,1.441984,0.039918,0.005187,0.063161,0.175741,...,0.000077,0.000075,0.000062,0.000069,0.000037,0.000038,0.000035,0.000041,0.000078,0.000062
540855,0.000014,0.000024,0.000029,0.000023,0.000032,0.000033,0.000019,0.000019,0.00002,0.000039,...,0.000004,0.000005,0.000007,0.000005,0.000005,0.000004,0.000004,0.000005,0.000004,0.000005
539275,56.517738,55.578827,39.456982,52.146332,187.437668,187.951538,7.107272,0.021488,0.018619,0.016941,...,0.000779,0.000749,0.005815,0.041543,0.046001,0.060624,0.01135,0.000467,0.000496,0.000455


In [461]:
def load_spherical_kmeans_features(file):
    ''' '''
    
    if run_on_colab == True:
        skmeans_features = np.load('/content/content/MyDrive/bird-songs/audio_8sec/' + file)
    else:
        skmeans_features = np.load(file)
    
    # Create df to hold sperical k means features
    df = pd.DataFrame(data = skmeans_features,
                      columns = ['skmeanfeat' + str(item+1) for item in list(range(skmeans_features.shape[1]))], 
                      index=df_xeno_canto.index)

    return df


df_skmean = load_spherical_kmeans_features('spherical_kmeans_features.npy')
df_skmean.head()

,skmeanfeat1,skmeanfeat2,skmeanfeat3,skmeanfeat4,skmeanfeat5,skmeanfeat6,skmeanfeat7,skmeanfeat8,skmeanfeat9,skmeanfeat10,...,skmeanfeat141,skmeanfeat142,skmeanfeat143,skmeanfeat144,skmeanfeat145,skmeanfeat146,skmeanfeat147,skmeanfeat148,skmeanfeat149,skmeanfeat150
fileid,,,,,,,,,,,,,,,,,,,,,
564895,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.000000,0.011965,0.008187,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0
545775,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0281,...,0.023483,0.000000,0.000000,0.117046,0.0,0.0,0.09548,0.0,0.126887,0.0
540857,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.000000,0.010672,0.009374,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0
540855,0.025926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.000000,0.010430,0.007600,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0
539275,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.000000,0.012091,0.004434,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0


In [494]:
def create_df_umap(df1, df2, df3):

    df_umap_step1 = df1.merge(df2, how ='left', left_index=True, right_index=True)
    df_umap_step2 = df_umap_step1.merge(df3, how ='left', left_index=True, right_index=True)
    return df_umap_step2


df_umap = create_df_umap(df_xeno_canto, df_mfcc, df_mel)

if run_on_colab == True:
    outfile = '/content/content/MyDrive/bird-songs/df_umap.pkl'
else:
    outfile = './df_umap.pkl'

df_umap.to_pickle(outfile)

display(df_umap.head())

,en,label,filename,start,end,rms,mfcc_max1,mfcc_max2,mfcc_max3,mfcc_max4,...,mel_std23,mel_std24,mel_std25,mel_std26,mel_std27,mel_std28,mel_std29,mel_std30,mel_std31,mel_std32
fileid,,,,,,,,,,,,,,,,,,,,,
564895,Carolina Wren,3,audio_8sec/564895.wav,70560.0,79381.0,15.446119,-149.43631,85.261353,15.307281,69.582359,...,0.056403,0.014043,0.017037,0.021666,0.038108,0.002578,0.002215,0.002122,0.002095,0.002019
545775,Carolina Wren,3,audio_8sec/545775.wav,79380.0,88201.0,17.320824,-34.174389,287.955383,19.568802,55.773628,...,0.051102,0.019693,0.004096,0.00468,0.005511,0.003314,0.003172,0.003048,0.002964,0.002909
540857,Carolina Wren,3,audio_8sec/540857.wav,8820.0,17641.0,6.876124,-307.844696,70.258423,39.420944,10.251158,...,0.000077,0.000075,0.000062,0.000069,0.000037,0.000038,0.000035,0.000041,0.000078,0.000062
540855,Carolina Wren,3,audio_8sec/540855.wav,158760.0,167581.0,14.775598,-559.795593,8.064632,12.457974,19.946301,...,0.000004,0.000005,0.000007,0.000005,0.000005,0.000004,0.000004,0.000005,0.000004,0.000005
539275,Carolina Wren,3,audio_8sec/539275.wav,132300.0,141121.0,12.824493,-217.228653,150.939987,180.321899,92.488434,...,0.000779,0.000749,0.005815,0.041543,0.046001,0.060624,0.01135,0.000467,0.000496,0.000455


In [495]:
def create_umap_chart(df, color_feature='label'):
    
    
    num_mfcc_features = NUMBER_MFCC*2
    num_mel_features = NUMBER_MEL*2
    
    umap_2d = UMAP(n_components=2, init='random', random_state=42)
    umap_2d.fit(df.iloc[:, 5:].values)
    
    projections = umap_2d.transform(df.iloc[:, 5:].values)
    
    fig = px.scatter(
        projections, x=0, y=1,
        color=list(df[color_feature].astype(str)), labels={'color': color_feature},
        title = f"UMAP Plot of bird audio samples colored by {color_feature}"
    )
    
    return fig


if run_on_colab == True:
    infile = '/content/content/MyDrive/bird-songs/df_umap.pkl'
else:
    infile = 'df_umap.pkl'

df_umap = pd.read_pickle(infile)


create_umap_chart(df_umap, color_feature='en')


In [496]:
def create_tsne_chart(df, color_feature='label'):
    
    
    num_mfcc_features = NUMBER_MFCC*2
    num_mel_features = NUMBER_MEL*2
    
    tsne = TSNE(n_components=2, random_state=42)
    
    projections = tsne.fit_transform(df.iloc[:, 5:].values)
    
    fig = px.scatter(
        projections, x=0, y=1,
        color=list(df[color_feature].astype(str)), labels={'color': color_feature},
        title = f"T-SNE Plot of bird audio samples colored by {color_feature}"
    )
    
    return fig


if run_on_colab == True:
    infile = '/content/content/MyDrive/bird-songs/df_umap.pkl'
else:
    infile = 'df_umap.pkl'

df_umap = pd.read_pickle(infile)


create_tsne_chart(df_umap, color_feature='en')

